# Scenario 3
## API to sqlite

![Scenario 3](../src/input/static/scenario_3.jpg)

In [1]:
# Import packages
import pandas as pd
import requests
import yaml
from sqlalchemy import create_engine

In [2]:
# Function to create payload
def create_payload(country,start,end):
    parameter = {
        'country_code': country,
        'start_date': start,
        'end_date': end
    }
    
    return parameter

In [3]:
# Load credential data from YAML-File
with open("../credentials.yml", "r") as credentials:
    try:
        credentials_data = yaml.safe_load(credentials)
        key = credentials_data['rapidsAPI']['API-KEY']
        host = credentials_data['rapidsAPI']['API-HOST']
    except:
        print('Error')

In [4]:
# Set variables for API call
url = 'https://working-days.p.rapidapi.com/1.3/analyse'

headers = {
    "X-RapidAPI-Key": key,
    "X-RapidAPI-Host": host
}

payload = create_payload('DE','2013-01-01','2013-01-31')

In [5]:
# Make API call and change response to JSON-Format
response = requests.get(url, headers=headers, params=payload)
jsn_response = response.json()

In [6]:
# Extract List of public holidays from JSON
lst_public_holidays = jsn_response['public_holidays']['list']

# Create DataFrame from List
df_public_holidays = pd.DataFrame(lst_public_holidays)

In [7]:
# Check DataFrame
df_public_holidays

,date,description
0,2013-01-01,Neujahrstag


### Dataframe To sqlite

In [8]:
# Write DataFrame to sqlite db
engine = create_engine('sqlite:///../src/output/workingDays', echo=False)
db_response = df_public_holidays.to_sql(con=engine, name="public_holidays", index=False, if_exists="replace")
db_response

1